# Loss Functions

In [24]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.linear_model import SGDRegressor, LinearRegression


In this exercise, you will compare the effects of Loss functions on a `LinearRegression` model.

👇 Let's download a CSV file to use for this challenge and parse it into a DataFrame

In [25]:
import pandas as pd

data = pd.read_csv("https://wagon-public-datasets.s3.amazonaws.com/05-Machine-Learning/04-Under-the-Hood/loss_functions_dataset.csv")
data.sample(5)


,Relative Compactness,Surface Area,Wall Area,Roof Area,Overall Height,Glazing Area,Average Temperature
521,0.64,784.0,343.0,220.5,3.5,0.25,18.435
36,0.66,759.5,318.5,220.5,3.5,0.00,9.790
362,0.74,686.0,245.0,220.5,3.5,0.25,13.670
654,0.71,710.5,269.5,220.5,3.5,0.40,15.320
599,0.76,661.5,416.5,122.5,7.0,0.40,40.060


🎯 Your task is to predict the average temperature inside a greenhouse based on its design. Your temperature predictions will help you select the appropriate greenhouse design for each plant, based on their climate needs. 

🌿 You know that plants can handle small temperature variations, but are exponentially more sensitive as the temperature variations increase. 

## 1. Theory 

❓ Theoretically, which Loss function would you train your model on to limit the risk of killing plants?

<details>
<summary> 🆘 Answer </summary>
    
By theory, you would use a Mean Square Error (MSE) Loss function. It would penalize outlier predictions and prevent your model from committing large errors. This would ensure smaller temperature variations and a lower risk for plants.

</details>

use a mean square error (mse) loss function to minimize outlier predictions and ensure stable temperatures, reducing the risk to plants.

## 2. Application

### 2.1 Preprocessing

❓ Standardise the features

In [26]:
X = data.drop('Average Temperature', axis=1)
y = data['Average Temperature']

scaler = StandardScaler()

X_scaled = scaler.fit_transform(X)


### 2.2 Modeling

In this section, you are going to verify the theory by evaluating models optimized on different Loss functions.

### Least Squares (MSE) Loss

❓ **10-Fold Cross-validate** a Linear Regression model optimized by **Stochastic Gradient Descent** (SGD) on a **Least Squares Loss** (MSE)



In [27]:
lr = LinearRegression()
sgdr = SGDRegressor(loss='squared_error')
cv = cross_validate(sgdr, X_scaled, y, cv = 5, scoring = ['max_error', 'r2'])
cv


{'fit_time': array([0.00344205, 0.00265408, 0.00272799, 0.00294495, 0.00259399]),
 'score_time': array([0.00058889, 0.00055122, 0.00047779, 0.00046992, 0.00053978]),
 'test_max_error': array([-9.86142927, -9.47437327, -8.97622685, -8.76745718, -8.10388467]),
 'test_r2': array([0.83891076, 0.88551226, 0.9160794 , 0.92192004, 0.91465609])}

❓ Compute 
- the mean cross-validated R2 score and save it in the variable `r2`
- the single biggest prediction error in °C of all your folds and save it in the variable `max_error_celsius`?

(Tips: `max_error` is an accepted scoring metric in sklearn)

In [28]:
scaler = StandardScaler()

# Fit and transform the features
X_scaled = scaler.fit_transform(X)

# Initialize the models
lr = LinearRegression()
sgdr = SGDRegressor(loss='squared_error')

# Compute cross-validated R2 score and max error
cv = cross_validate(sgdr, X_scaled, y, cv=5, scoring=['r2', 'max_error'])

# Extract the results
r2 = cv['test_r2'].mean()
max_error_celsius = cv['test_max_error'].max()

print("mean cross-validated r2 score:", r2)
print("single biggest prediction error in °C:", max_error_celsius)


mean cross-validated r2 score: 0.8950105108420734
single biggest prediction error in °C: -8.091905878939137


### Mean Absolute Error (MAE) Loss

What if we optimize our model on the MAE instead?

❓ **10-Fold Cross-validate** a Linear Regression model optimized by **Stochastic Gradient Descent** (SGD) on a **MAE** Loss

<details>
<summary>💡 Hints</summary>

- MAE loss cannot be directly specified in `SGDRegressor`. It must be engineered by adjusting the right parameters

</details>

In [29]:
scaler = StandardScaler()

X_scaled = scaler.fit_transform(X)

sgdr_mae = SGDRegressor(loss='epsilon_insensitive', epsilon=0, max_iter=1000, tol=1e-3)

mae_scores = cross_val_score(sgdr_mae, X_scaled, y, cv=10, scoring='neg_mean_absolute_error')

mean_mae = -np.mean(mae_scores)

print("mean mae score:", mean_mae)


mean mae score: 2.289565917039359


❓ Compute 
- the mean cross-validated R2 score, store it in `r2_mae`
- the single biggest prediction error of all your folds, store it in `max_error_mae`?

In [30]:
scaler = StandardScaler()

X_scaled = scaler.fit_transform(X)

sgdr_mae = SGDRegressor(loss='epsilon_insensitive', epsilon=0, max_iter=1000, tol=1e-3)

cv_mae = cross_validate(sgdr_mae, X_scaled, y, cv=10, scoring=['r2', 'max_error'])

r2_mae = cv_mae['test_r2'].mean()
max_error_mae = -cv_mae['test_max_error'].max()

print("mean cross-validated r2 score:", r2_mae)
print("single biggest prediction error in °C:", max_error_mae)


mean cross-validated r2 score: 0.8762611003352134
single biggest prediction error in °C: 10.132856845452338


## 3. Conclusion

❓Which of the models you evaluated seems the most appropriate for your task?

<details>
<summary> 🆘Answer </summary>
    
Although mean cross-validated r2 scores are approximately similar between the two models, the one optimized on a MAE has more chance to make larger mistakes from time to time, increasing the risk of killing plants!

    
</details>

mean squared error (mse) loss

# 🏁 Check your code and push your notebook

In [31]:
from nbresult import ChallengeResult

result = ChallengeResult(
    'loss_functions',
    r2 = r2,
    r2_mae = r2_mae,
    max_error = max_error_celsius,
    max_error_mae = max_error_mae
)

result.write()
print(result.check())



============================= test session starts ==============================
platform darwin -- Python 3.10.6, pytest-7.1.3, pluggy-1.0.0 -- /Users/reecepalmer/.pyenv/versions/3.10.6/envs/lewagon/bin/python3
cachedir: .pytest_cache
rootdir: /Users/reecepalmer/Code/RPalmr/05-ML/04-Under-the-hood/data-loss-functions/tests
plugins: asyncio-0.19.0, dash-2.14.0, typeguard-2.13.3, anyio-3.6.2, hydra-core-1.3.2
asyncio: mode=strict
collecting ... collected 3 items

test_loss_functions.py::TestLossFunctions::test_max_error_order PASSED   [ 33%]
test_loss_functions.py::TestLossFunctions::test_r2 PASSED                [ 66%]
test_loss_functions.py::TestLossFunctions::test_r2_mae PASSED            [100%]

============================== 3 passed in 0.06s ===============================


💯 You can commit your code:

git add tests/loss_functions.pickle

git commit -m 'Completed loss_functions step'

git push origin master

